In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

In [7]:
reduced_books_users_ratings = pd.read_csv("data/clean/reduced_books_users_ratings.csv", index_col=0)
books_users_ratings = pd.read_csv("data/clean/books_users_ratings.csv", index_col=0)

In [10]:
reduced_books_users_ratings.head()
books_users_ratings.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn,Unnamed: 0.1,age,city,state,country
0,276726,0155061224,5,Rites of Passage,Judith Rae,2001.0,Heinle,0155061224,276725,34.786876,seattle,washington,usa
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999.0,Cambridge University Press,052165615X,276728,16.000000,rijeka,n/a,croatia
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001.0,Cambridge University Press,0521795028,276728,16.000000,rijeka,n/a,croatia
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001.0,Doubleday,038550120X,276743,34.786876,torrance,california,usa
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,2001.0,Doubleday,038550120X,11675,34.786876,NaN,n/a,n/a
